In [1]:
import time
import pandas as pd
import numpy as np

CITY_DATA = {
    'chicago': 'chicago.csv',
    'new york city': 'newyorkcity.csv',
    'washington': 'washington.csv'
}

def get_filters():
    """
    Asks the user to specify a city, month, and day to analyze.

    Returns:
        (str) city - name of the city to analyze
        (str) month - name of the month to filter by, or "all" to apply no month filter
        (str) day - name of the day of the week to filter by, or "all" to apply no day filter
    """
    print('Hello! Let\'s explore some US bikeshare data!')
    
    # get user input for city (chicago, new york city, washington)
    while True:
        cities = ['chicago', 'new york city', 'washington']
        city = input("Which city would you like to analyze? (Chicago, New York City, Washington)\n").lower()
        if city in cities:
            break
        else:
            print("Please check your spelling and re-enter one of the three cities")
    
    # get user input for month (january, february, ... , june)
    while True:
        months = ['January', 'February', 'March', 'April', 'May', 'June', 'All']
        month = input("Which month would you like to analyze? Enter the full month name or 'All' for all 6 months.\n").title()
        if month in months:
            break
        else:
            print("Please enter a valid month")
    
    # get user input for day of the week (Monday, Tuesday, ... Sunday)
    while True:
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'All']
        day = input("Which day of the week would you like to analyze? Enter the full day name or 'All' for all 7 days.\n").title()
        if day in days:
            break
        else:
            print("Please check your spelling and re-enter a valid day...")
    
    print('-' * 40)
    return city, month, day

def load_data(city, month, day):
    """
    Loads data for the specified city and filters by month and day if applicable.

    Args:
        (str) city - name of the city to analyze
        (str) month - name of the month to filter by, or "all" to apply no month filter
        (str) day - name of the day of the week to filter by, or "all" to apply no day filter
    Returns:
        bikeshare - Pandas DataFrame containing city data filtered by month and day
    """
    # load data file into a dataframe
    bikeshare = pd.read_csv(CITY_DATA[city])

    # convert the Start Time column to datetime
    bikeshare['Start Time'] = pd.to_datetime(bikeshare['Start Time'])
    bikeshare['month'] = bikeshare['Start Time'].dt.month
    bikeshare['day_of_week'] = bikeshare['Start Time'].dt.day_name()
    
    if month != 'All':                # filter by month 
        months = ['January', 'February', 'March', 'April', 'May', 'June']
        month = months.index(month) + 1
        bikeshare = bikeshare[bikeshare['month'] == month] 

    if day != 'All':                  # filter by weekday
        bikeshare = bikeshare[bikeshare['day_of_week'] == day]

    return bikeshare

def time_stats(bikeshare, month, day):
    """Displays statistics on the most frequent times of travel."""
    print('\nCalculating The Most Frequent Times of Travel...\n')
    start_time = time.time()

    if month == 'All':
        most_popular_month = bikeshare['month'].mode()[0]
        months = ['January', 'February', 'March', 'April', 'May', 'June']
        most_popular_month = months[most_popular_month - 1]
        print("The most common month is", most_popular_month)

    if day == 'All':
        most_popular_day = bikeshare['day_of_week'].mode()[0]
        print("The most common week day is", most_popular_day)

    bikeshare['Start Hour'] = bikeshare['Start Time'].dt.hour
    most_popular_hour = bikeshare['Start Hour'].mode()[0]
    print("The most common Start Hour is {} hrs".format(most_popular_hour))
    
    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)

def station_stats(bikeshare):
    """Displays statistics on the most popular stations and trip."""
    print('\nCalculating The Most Popular Stations and Trip...\n')
    start_time = time.time()

    # display most commonly used start station
    most_popular_start_station = bikeshare['Start Station'].mode()[0]
    print("The most commonly used Start Station is {}".format(most_popular_start_station))

    # display most commonly used end station
    most_popular_end_station = bikeshare['End Station'].mode()[0]
    print("The most commonly used End Station is {}".format(most_popular_end_station))

    # display most frequent combination of start station and end station trip
    bikeshare['combination'] = "(" + bikeshare['Start Station'] + ")"+ " - " + "(" + bikeshare['End Station'] +")"
    most_popular_combination = bikeshare['combination'].mode()[0]
    print("The most frequent combination of Start and End Station Trip is {} ".format(most_popular_combination))

    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)

def trip_duration_stats(bikeshare):
    """Displays statistics on the total and average trip duration."""
    print('\nCalculating Trip Duration...\n')
    start_time = time.time()

    # display total travel time
    trip_duration_sum = bikeshare['Trip Duration'].sum()
    minute, second = divmod(trip_duration_sum, 60)
    hour, minute = divmod(minute, 60)
    print("The total trip duration is {} hours {} minutes {} seconds".format(hour, minute, second))
    
    # display mean travel time
    mean_travel_time = bikeshare['Trip Duration'].mean()
    rounded_mean_travel_time = round(mean_travel_time, 2)  # Round to 2 decimal places
    hours, remainder = divmod(rounded_mean_travel_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print('\nThe mean travel time is {} hours {} minutes {} seconds'.format(hours, minutes, seconds))
    
    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)

def user_stats(bikeshare, city):
    """Displays statistics on bikeshare users."""
    print('\nCalculating User Stats...\n')
    start_time = time.time()

    # Display counts of user types
    user_counts = bikeshare['User Type'].value_counts()
    print("The user types are:\n", user_counts)
    
    # Display counts of gender if the 'Gender' column exists
    if 'Gender' in bikeshare:
        gender_count = bikeshare['Gender'].value_counts()
        print('\nCount of Gender:\n{}\n'.format(gender_count))
    else:
        print('\nCount of Gender could not be displayed; the dataset does not contain information for Gender\n')

    # Display earliest, most recent, and most common year of birth if 'Birth Year' exists
    if 'Birth Year' in bikeshare:
        earliest_dob = int(bikeshare['Birth Year'].min())
        most_recent_dob = int(bikeshare['Birth Year'].max())
        most_common_dob = int(bikeshare['Birth Year'].mode()[0])

        print('Birth Year Statistics:')
        print('Earliest Date of Birth: {}'.format(earliest_dob))
        print('Most Recent Year of Birth: {}'.format(most_recent_dob))
        print('Most Common Year of Birth: {}'.format(most_common_dob))
    else:
        print('\nBirth Year statistics could not be displayed; the dataset does not contain information for Birth Year\n')

    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)
    
def summary(bikeshare):
    """Displays statistics on the dataset."""
    print('\nCheck 5 point summary...\n')
    start_time = time.time()
    
    data_summary =  bikeshare.describe().transpose()
    print('\n5 point summary: {}\n'.format(data_summary))

    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)
    
def display_dataset(bikeshare):
    """Displays 5 rows of data."""
    print('\nLet us display some sample of our data..\n')
    while True:
        raw = input('Would you like to see 5 rows of the dataset? Please enter yes or no: ').lower()
        if raw == 'yes':
            i = 5
            print('First {} rows:'.format(i))
            result = bikeshare.head(i)
            print(result)

            sample_bike_data = input('\nWould you like to see 5 more rows of the dataset? Please enter yes or no:\n').lower()
            i += 5
            if sample_bike_data != 'yes':
                break
        elif raw == 'no':
            break
        else:
            raw = input("\nYour input is invalid. Please enter only 'yes' or 'no'\n").lower()


def main():
    while True:
        city, month, day = get_filters()
        bikeshare = load_data(city, month, day)

        time_stats(bikeshare, month, day)
        station_stats(bikeshare)
        trip_duration_stats(bikeshare)
        user_stats(bikeshare, city)
        summary(bikeshare)
        display_dataset(bikeshare)
        
        restart = input('\nWould you like to restart? Enter yes or no.\n')
        if restart.lower() != 'yes':
            break

if __name__ == "__main__":
    main()

Hello! Let's explore some US bikeshare data!
Which city would you like to analyze? (Chicago, New York City, Washington)
chicago
Which month would you like to analyze? Enter the full month name or 'All' for all 6 months.
all
Which day of the week would you like to analyze? Enter the full day name or 'All' for all 7 days.
all
----------------------------------------

Calculating The Most Frequent Times of Travel...

The most common month is June
The most common week day is Tuesday
The most common Start Hour is 17 hrs

This took 0.08350729942321777 seconds.
----------------------------------------

Calculating The Most Popular Stations and Trip...

The most commonly used Start Station is Streeter Dr & Grand Ave
The most commonly used End Station is Streeter Dr & Grand Ave
The most frequent combination of Start and End Station Trip is (Lake Shore Dr & Monroe St) - (Streeter Dr & Grand Ave) 

This took 0.4121890068054199 seconds.
----------------------------------------

Calculating Trip Du